# Tensorboard(TensorFlow版本)

In [1]:
import tensorflow as tf
import numpy as np
tf.compat.v1.reset_default_graph()

x = np.linspace(0, 2*np.pi, 100)
y = np.cos(x)

summary_writer = tf.summary.create_file_writer('tensorboard_log')

with summary_writer.as_default():
    for i, j in enumerate(y):
        tf.summary.scalar('cos', j, step=i)

In [ ]:
# 測試的時候先到command window輸入 tensorboard --logdir tensorboard_log  ... 儲存位置
# 如果上述指令出問題，可以先移除google-auth再安裝google-auth
# pip uninstall google-auth
# pip install google-auth==1.6.3

In [5]:
import tensorflow as tf
import datetime
tf.compat.v1.reset_default_graph()


mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])


model = create_model()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

# 用decorator加速底層程式碼
@tf.function
def train_step(x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)

@tf.function
def test_step(x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)


current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs4/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs4/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
EPOCHS = 5

%timeit
for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()



Epoch 1, Loss: 0.24588598310947418, Accuracy: 92.92166900634766, Test Loss: 0.1144426167011261, Test Accuracy: 96.66999816894531
Epoch 2, Loss: 0.10631169378757477, Accuracy: 96.83999633789062, Test Loss: 0.0945863202214241, Test Accuracy: 97.1500015258789
Epoch 3, Loss: 0.07340969890356064, Accuracy: 97.71833801269531, Test Loss: 0.07232426851987839, Test Accuracy: 97.80999755859375
Epoch 4, Loss: 0.05694334954023361, Accuracy: 98.1866683959961, Test Loss: 0.06694125384092331, Test Accuracy: 97.83999633789062
Epoch 5, Loss: 0.043302539736032486, Accuracy: 98.64833068847656, Test Loss: 0.062267035245895386, Test Accuracy: 98.02999877929688
